In [1]:
import numpy as np
import pandas as pd

In [2]:
df_comed = pd.read_csv("resources/COMED_hourly.csv")

In [3]:
df_comed.head()

,Datetime,COMED_MW
0,2011-12-31 01:00:00,9970.0
1,2011-12-31 02:00:00,9428.0
2,2011-12-31 03:00:00,9059.0
3,2011-12-31 04:00:00,8817.0
4,2011-12-31 05:00:00,8743.0


In [4]:
df_comed.shape

(66497, 2)

In [5]:
df_comed.dtypes

Datetime     object
COMED_MW    float64
dtype: object